In [11]:
import cv2
import pylab as plt
import numpy as np

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

# ---------- PARAMETERS ----------
dot_density = 4  # larger = fewer dots (controls downscaling)
dot_scale = 1.0  # base size multiplier for dots (0.5-2.0 recommended)
alpha_value = 0.40  # transparency of dots
output_file = "Art_v2_improvednobgfinal40.png"
use_adaptive_sizing = True  # size dots based on local detail
use_voronoi = False  # set True for more artistic, organic spacing

# ---------- LOAD IMAGE ----------
img = cv2.imread("CJNOBG.png")  # load in color
if img is None:
    raise FileNotFoundError("CJ.png not found.")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # convert to RGB for plotting

h, w, _ = img.shape

# ---------- OPTIONAL CROPPING ----------
img_c = img[145:145+1080, :]  # adjust as needed
h_c, w_c, _ = img_c.shape

# ---------- RESIZE TO CONTROL DOT DENSITY ----------
img_r = cv2.resize(img_c, (w_c // dot_density, h_c // dot_density), interpolation=cv2.INTER_AREA)
h_r, w_r, _ = img_r.shape

# ---------- CONVERT TO GRAYSCALE FOR BRIGHTNESS ----------
gray = cv2.cvtColor(img_r, cv2.COLOR_RGB2GRAY)

# ---------- ADAPTIVE SIZING ----------
# Use edge detection to vary dot size based on image complexity
if use_adaptive_sizing:
    edges = cv2.Canny(gray, 50, 150)
    edges = cv2.GaussianBlur(edges, (5, 5), 0)
    # Normalize edges to 0.3-1.0 range for size variation
    size_map = 0.3 + (edges / 255.0) * 0.7
else:
    size_map = np.ones_like(gray, dtype=float)

# ---------- BUILD SCATTER DATA ----------
lX, lY, lS, lC = [], [], [], []

for i in range(h_r):
    for j in range(w_r):
        lX.append(i)
        lY.append(j)
        
        # Size based on brightness: darker areas get larger dots
        brightness = gray[i, j]
        relative_brightness = brightness / 255.0
        size = (1.0 - relative_brightness) * 0.8 + 0.2  # range 0.2 to 1.0
        
        # Apply adaptive sizing if enabled
        size *= size_map[i, j]
        size *= dot_scale
        
        lS.append(size)
        lC.append(img_r[i, j] / 255.0)  # RGB color normalized

# ---------- CONVERT LISTS TO ARRAYS ----------
lX = np.array(lX)
lY = np.array(lY)
lS = np.array(lS)
lC = np.array(lC)

# ---------- NORMALIZE SIZES FOR BETTER SCALING ----------
lS = 50 + (lS / lS.max()) * 150  # scale sizes to reasonable range (50-200)

# ---------- PLOT ----------
fig = plt.figure(figsize=(60, 60), dpi=100, facecolor='none')
ax = plt.subplot(111)
ax.patch.set_alpha(0)  # make axes background transparent

# Use scatter with better settings to avoid overlap
scatter = ax.scatter(
    lY, lX,
    s=lS,
    c=lC,
    alpha=alpha_value,
    edgecolors='none',
    marker='o',
    rasterized=True  # improves rendering efficiency
)

ax.set_aspect('equal')
ax.axis('off')
ax.invert_yaxis()  # keep orientation correct
plt.tight_layout(pad=0)

plt.savefig(output_file, dpi=300, bbox_inches='tight', pad_inches=0, facecolor='none', transparent=True)
plt.close()

print(f"Saved improved pointillism image as {output_file}")
print(f"Image dimensions: {w_r} x {h_r} dots")
print(f"Total dots: {len(lX)}")

Saved improved pointillism image as Art_v2_improvednobgfinal.png
Image dimensions: 148 x 139 dots
Total dots: 20572
